# 模型训练和评估验证

# 导入包

In [1]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
import time
import re

# 导入特征工程后的数据

In [2]:
inputfile = '../datasets/data_feature_processed.csv'
modelfile = '../model/xgb.pkl'

In [3]:
data = pd.read_csv(inputfile)

In [4]:
data[(data.loc_id==33)&(data.time_stamp>='2017-10-01')].shape

(744, 92)

# XGBoost模型训练

In [5]:
df = data.copy()
df.sort_values(by=['loc_id','time_stamp'],inplace=True)

In [6]:
df.head()

,time_stamp,loc_id,num_of_people,month_of_year,day_of_week,week_of_year,history_1h_population,history_2h_population,history_3h_population,history_4h_population,...,24,25,26,27,28,29,30,31,32,33
0,2017-03-01 00:00:00,1,49.0,3,2,9,71.0,66.0,51.0,60.0,...,0,0,0,0,0,0,0,0,0,0
33,2017-03-01 01:00:00,1,49.0,3,2,9,49.0,71.0,66.0,51.0,...,0,0,0,0,0,0,0,0,0,0
66,2017-03-01 02:00:00,1,54.0,3,2,9,49.0,49.0,71.0,66.0,...,0,0,0,0,0,0,0,0,0,0
99,2017-03-01 03:00:00,1,456.0,3,2,9,54.0,49.0,49.0,71.0,...,0,0,0,0,0,0,0,0,0,0
132,2017-03-01 04:00:00,1,470.0,3,2,9,456.0,54.0,49.0,49.0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df.tail()

,time_stamp,loc_id,num_of_people,month_of_year,day_of_week,week_of_year,history_1h_population,history_2h_population,history_3h_population,history_4h_population,...,24,25,26,27,28,29,30,31,32,33
193907,2017-10-31 19:00:00,33,407.0,10,1,44,919.0,1540.0,740.0,1127.0,...,0,0,0,0,0,0,0,0,0,1
193940,2017-10-31 20:00:00,33,496.0,10,1,44,407.0,919.0,1540.0,740.0,...,0,0,0,0,0,0,0,0,0,1
193973,2017-10-31 21:00:00,33,434.0,10,1,44,496.0,407.0,919.0,1540.0,...,0,0,0,0,0,0,0,0,0,1
194006,2017-10-31 22:00:00,33,272.0,10,1,44,434.0,496.0,407.0,919.0,...,0,0,0,0,0,0,0,0,0,1
194039,2017-10-31 23:00:00,33,120.0,10,1,44,272.0,434.0,496.0,407.0,...,0,0,0,0,0,0,0,0,0,1


## 1. 划分训练集和测试集

In [8]:
train = df
test = df[df.time_stamp>='2017-10-01']
train_X = train.iloc[:,3:]
train_y = train['num_of_people']
test_X = test.iloc[:,3:]
test_y = test['num_of_people']
print(train_X.shape,test_X.shape)

(194040, 89) (24552, 89)


## 2. XGBoost训练

In [14]:
model_xgb = xgboost_model_train(train_X,train_y,test_X,test_y)

[0]	train-rmse:683.794	test-rmse:943.305
Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.

Will train until test-rmse hasn't improved in 20 rounds.
[1]	train-rmse:664.798	test-rmse:917.844
[2]	train-rmse:646.655	test-rmse:893.43
[3]	train-rmse:628.851	test-rmse:869.608
[4]	train-rmse:611.565	test-rmse:846.271
[5]	train-rmse:594.828	test-rmse:823.843
[6]	train-rmse:578.705	test-rmse:802.377
[7]	train-rmse:563.032	test-rmse:781.385
[8]	train-rmse:547.741	test-rmse:760.791
[9]	train-rmse:532.965	test-rmse:740.93
[10]	train-rmse:518.662	test-rmse:721.722
[11]	train-rmse:504.786	test-rmse:703.148
[12]	train-rmse:491.548	test-rmse:685.309
[13]	train-rmse:478.531	test-rmse:667.836
[14]	train-rmse:465.978	test-rmse:650.907
[15]	train-rmse:453.79	test-rmse:634.564
[16]	train-rmse:442.182	test-rmse:618.614
[17]	train-rmse:431.024	test-rmse:603.507
[18]	train-rmse:419.908	test-rmse:588.452
[19]	train-rmse:409.228	test-rmse:574.123
[20]	train-rmse:398.808	test-r

[191]	train-rmse:105.895	test-rmse:135.716
[192]	train-rmse:105.791	test-rmse:135.475
[193]	train-rmse:105.681	test-rmse:135.311
[194]	train-rmse:105.601	test-rmse:135.15
[195]	train-rmse:105.502	test-rmse:135.009
[196]	train-rmse:105.434	test-rmse:134.887
[197]	train-rmse:105.365	test-rmse:134.759
[198]	train-rmse:105.301	test-rmse:134.527
[199]	train-rmse:105.218	test-rmse:134.302
[200]	train-rmse:105.137	test-rmse:134.189
[201]	train-rmse:105.012	test-rmse:134.011
[202]	train-rmse:104.954	test-rmse:133.921
[203]	train-rmse:104.891	test-rmse:133.826
[204]	train-rmse:104.85	test-rmse:133.734
[205]	train-rmse:104.755	test-rmse:133.512
[206]	train-rmse:104.712	test-rmse:133.413
[207]	train-rmse:104.65	test-rmse:133.331
[208]	train-rmse:104.583	test-rmse:133.201
[209]	train-rmse:104.508	test-rmse:133.071
[210]	train-rmse:104.416	test-rmse:132.92
[211]	train-rmse:104.355	test-rmse:132.775
[212]	train-rmse:104.32	test-rmse:132.704
[213]	train-rmse:104.26	test-rmse:132.585
[214]	train-rmse:

[383]	train-rmse:94.8194	test-rmse:116.999
[384]	train-rmse:94.7962	test-rmse:116.966
[385]	train-rmse:94.757	test-rmse:116.911
[386]	train-rmse:94.7042	test-rmse:116.845
[387]	train-rmse:94.6468	test-rmse:116.759
[388]	train-rmse:94.6172	test-rmse:116.724
[389]	train-rmse:94.5382	test-rmse:116.612
[390]	train-rmse:94.4872	test-rmse:116.541
[391]	train-rmse:94.4241	test-rmse:116.458
[392]	train-rmse:94.4076	test-rmse:116.422
[393]	train-rmse:94.336	test-rmse:116.308
[394]	train-rmse:94.2833	test-rmse:116.232
[395]	train-rmse:94.213	test-rmse:116.127
[396]	train-rmse:94.1667	test-rmse:116.074
[397]	train-rmse:94.1192	test-rmse:116.025
[398]	train-rmse:94.0824	test-rmse:115.97
[399]	train-rmse:94.0179	test-rmse:115.918
[400]	train-rmse:93.9918	test-rmse:115.874
[401]	train-rmse:93.9818	test-rmse:115.85
[402]	train-rmse:93.9224	test-rmse:115.769
[403]	train-rmse:93.8771	test-rmse:115.708
[404]	train-rmse:93.8236	test-rmse:115.632
[405]	train-rmse:93.7906	test-rmse:115.598
[406]	train-rmse

[575]	train-rmse:87.5334	test-rmse:105.467
[576]	train-rmse:87.4749	test-rmse:105.372
[577]	train-rmse:87.4311	test-rmse:105.309
[578]	train-rmse:87.4108	test-rmse:105.274
[579]	train-rmse:87.3954	test-rmse:105.251
[580]	train-rmse:87.352	test-rmse:105.184
[581]	train-rmse:87.3221	test-rmse:105.138
[582]	train-rmse:87.2643	test-rmse:105.037
[583]	train-rmse:87.2418	test-rmse:105.014
[584]	train-rmse:87.1792	test-rmse:104.886
[585]	train-rmse:87.1742	test-rmse:104.871
[586]	train-rmse:87.1376	test-rmse:104.782
[587]	train-rmse:87.11	test-rmse:104.75
[588]	train-rmse:87.058	test-rmse:104.673
[589]	train-rmse:87.037	test-rmse:104.654
[590]	train-rmse:86.9989	test-rmse:104.574
[591]	train-rmse:86.9342	test-rmse:104.505
[592]	train-rmse:86.901	test-rmse:104.464
[593]	train-rmse:86.8729	test-rmse:104.326
[594]	train-rmse:86.8241	test-rmse:104.28
[595]	train-rmse:86.8018	test-rmse:104.258
[596]	train-rmse:86.7616	test-rmse:104.221
[597]	train-rmse:86.7074	test-rmse:104.174
[598]	train-rmse:86

[767]	train-rmse:82.2123	test-rmse:97.053
[768]	train-rmse:82.1999	test-rmse:97.0307
[769]	train-rmse:82.1737	test-rmse:96.9906
[770]	train-rmse:82.1458	test-rmse:96.9615
[771]	train-rmse:82.1233	test-rmse:96.931
[772]	train-rmse:82.0885	test-rmse:96.8911
[773]	train-rmse:82.0726	test-rmse:96.8687
[774]	train-rmse:82.0651	test-rmse:96.853
[775]	train-rmse:82.0383	test-rmse:96.8323
[776]	train-rmse:82.0099	test-rmse:96.7932
[777]	train-rmse:81.9838	test-rmse:96.7596
[778]	train-rmse:81.9651	test-rmse:96.737
[779]	train-rmse:81.9393	test-rmse:96.6928
[780]	train-rmse:81.9014	test-rmse:96.6388
[781]	train-rmse:81.8889	test-rmse:96.6191
[782]	train-rmse:81.8646	test-rmse:96.5871
[783]	train-rmse:81.8312	test-rmse:96.4046
[784]	train-rmse:81.8101	test-rmse:96.3786
[785]	train-rmse:81.7738	test-rmse:96.3382
[786]	train-rmse:81.7355	test-rmse:96.2951
[787]	train-rmse:81.7223	test-rmse:96.2802
[788]	train-rmse:81.704	test-rmse:96.2445
[789]	train-rmse:81.6863	test-rmse:96.2272
[790]	train-rmse

...

[1148]	train-rmse:73.8762	test-rmse:84.5112
[1149]	train-rmse:73.8497	test-rmse:84.4795
[1150]	train-rmse:73.8261	test-rmse:84.4471
[1151]	train-rmse:73.8159	test-rmse:84.4319
[1152]	train-rmse:73.7875	test-rmse:84.371
[1153]	train-rmse:73.7739	test-rmse:84.3491
[1154]	train-rmse:73.7629	test-rmse:84.3327
[1155]	train-rmse:73.7418	test-rmse:84.3147
[1156]	train-rmse:73.7196	test-rmse:84.2765
[1157]	train-rmse:73.7119	test-rmse:84.2653
[1158]	train-rmse:73.6932	test-rmse:84.2428
[1159]	train-rmse:73.6691	test-rmse:84.2091
[1160]	train-rmse:73.6528	test-rmse:84.187
[1161]	train-rmse:73.637	test-rmse:84.1687
[1162]	train-rmse:73.615	test-rmse:84.1389
[1163]	train-rmse:73.5881	test-rmse:84.1062
[1164]	train-rmse:73.5762	test-rmse:84.09
[1165]	train-rmse:73.5481	test-rmse:84.0466
[1166]	train-rmse:73.5427	test-rmse:84.0414
[1167]	train-rmse:73.5309	test-rmse:84.0096
[1168]	train-rmse:73.5047	test-rmse:83.9724
[1169]	train-rmse:73.497	test-rmse:83.9608
[1170]	train-rmse:73.4692	test-rmse:83.

[1336]	train-rmse:70.4586	test-rmse:79.3227
[1337]	train-rmse:70.434	test-rmse:79.2868
[1338]	train-rmse:70.4159	test-rmse:79.2434
[1339]	train-rmse:70.4064	test-rmse:79.2355
[1340]	train-rmse:70.3972	test-rmse:79.2227
[1341]	train-rmse:70.3738	test-rmse:79.1851
[1342]	train-rmse:70.3496	test-rmse:79.1595
[1343]	train-rmse:70.3231	test-rmse:79.1258
[1344]	train-rmse:70.3151	test-rmse:79.1168
[1345]	train-rmse:70.2983	test-rmse:79.0993
[1346]	train-rmse:70.285	test-rmse:79.0861
[1347]	train-rmse:70.2664	test-rmse:79.0681
[1348]	train-rmse:70.2547	test-rmse:79.0515
[1349]	train-rmse:70.2299	test-rmse:79.015
[1350]	train-rmse:70.2152	test-rmse:78.997
[1351]	train-rmse:70.1944	test-rmse:78.9604
[1352]	train-rmse:70.1687	test-rmse:78.9203
[1353]	train-rmse:70.1475	test-rmse:78.884
[1354]	train-rmse:70.1207	test-rmse:78.846
[1355]	train-rmse:70.1166	test-rmse:78.8401
[1356]	train-rmse:70.0893	test-rmse:78.7993
[1357]	train-rmse:70.0612	test-rmse:78.7558
[1358]	train-rmse:70.046	test-rmse:78.

[1524]	train-rmse:67.2644	test-rmse:74.5599
[1525]	train-rmse:67.2576	test-rmse:74.5534
[1526]	train-rmse:67.2389	test-rmse:74.5388
[1527]	train-rmse:67.2153	test-rmse:74.5049
[1528]	train-rmse:67.2021	test-rmse:74.4912
[1529]	train-rmse:67.1928	test-rmse:74.4764
[1530]	train-rmse:67.175	test-rmse:74.4549
[1531]	train-rmse:67.1492	test-rmse:74.417
[1532]	train-rmse:67.1278	test-rmse:74.3858
[1533]	train-rmse:67.1136	test-rmse:74.3656
[1534]	train-rmse:67.0898	test-rmse:74.32
[1535]	train-rmse:67.085	test-rmse:74.3131
[1536]	train-rmse:67.0774	test-rmse:74.3049
[1537]	train-rmse:67.0591	test-rmse:74.2886
[1538]	train-rmse:67.0325	test-rmse:74.2486
[1539]	train-rmse:67.029	test-rmse:74.2432
[1540]	train-rmse:67.0197	test-rmse:74.2308
[1541]	train-rmse:67.0058	test-rmse:74.2158
[1542]	train-rmse:66.9888	test-rmse:74.1875
[1543]	train-rmse:66.9737	test-rmse:74.169
[1544]	train-rmse:66.9561	test-rmse:74.148
[1545]	train-rmse:66.9367	test-rmse:74.1238
[1546]	train-rmse:66.9295	test-rmse:74.1

[1712]	train-rmse:64.3581	test-rmse:70.367
[1713]	train-rmse:64.3393	test-rmse:70.3477
[1714]	train-rmse:64.3208	test-rmse:70.3318
[1715]	train-rmse:64.3082	test-rmse:70.314
[1716]	train-rmse:64.2921	test-rmse:70.2931
[1717]	train-rmse:64.2886	test-rmse:70.2885
[1718]	train-rmse:64.2752	test-rmse:70.2601
[1719]	train-rmse:64.2542	test-rmse:70.2344
[1720]	train-rmse:64.2357	test-rmse:70.2088
[1721]	train-rmse:64.2251	test-rmse:70.1969
[1722]	train-rmse:64.2095	test-rmse:70.1797
[1723]	train-rmse:64.2002	test-rmse:70.1568
[1724]	train-rmse:64.1907	test-rmse:70.1467
[1725]	train-rmse:64.1729	test-rmse:70.1189
[1726]	train-rmse:64.1589	test-rmse:70.0996
[1727]	train-rmse:64.1371	test-rmse:70.0646
[1728]	train-rmse:64.1181	test-rmse:70.0332
[1729]	train-rmse:64.0961	test-rmse:70.002
[1730]	train-rmse:64.0797	test-rmse:69.9836
[1731]	train-rmse:64.0612	test-rmse:69.9644
[1732]	train-rmse:64.0425	test-rmse:69.9473
[1733]	train-rmse:64.0278	test-rmse:69.9292
[1734]	train-rmse:64.0121	test-rmse

[1899]	train-rmse:61.6964	test-rmse:66.7433
[1900]	train-rmse:61.687	test-rmse:66.7324
[1901]	train-rmse:61.6746	test-rmse:66.6841
[1902]	train-rmse:61.6538	test-rmse:66.6619
[1903]	train-rmse:61.6381	test-rmse:66.6337
[1904]	train-rmse:61.6179	test-rmse:66.6133
[1905]	train-rmse:61.6047	test-rmse:66.5923
[1906]	train-rmse:61.5898	test-rmse:66.5711
[1907]	train-rmse:61.5777	test-rmse:66.5597
[1908]	train-rmse:61.5582	test-rmse:66.5315
[1909]	train-rmse:61.5503	test-rmse:66.5215
[1910]	train-rmse:61.5295	test-rmse:66.4899
[1911]	train-rmse:61.5158	test-rmse:66.4733
[1912]	train-rmse:61.5066	test-rmse:66.4666
[1913]	train-rmse:61.5018	test-rmse:66.4625
[1914]	train-rmse:61.4871	test-rmse:66.449
[1915]	train-rmse:61.4675	test-rmse:66.408
[1916]	train-rmse:61.4465	test-rmse:66.3756
[1917]	train-rmse:61.4286	test-rmse:66.3506
[1918]	train-rmse:61.4071	test-rmse:66.3376
[1919]	train-rmse:61.3908	test-rmse:66.3201
[1920]	train-rmse:61.3833	test-rmse:66.3093
[1921]	train-rmse:61.3642	test-rmse

## 3. 模型评估

In [37]:
test_df = pd.read_csv('../datasets/test_data.csv',parse_dates=['time_stamp'])

In [38]:
%time test_df = real_time_predict(model_xgb,test_df)

/notebooks/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[161.]
第0行预测完成！
[107.]
第1行预测完成！
[89.]
第2行预测完成！
[84.]
第3行预测完成！
[84.]
第4行预测完成！
[91.]
第5行预测完成！
[129.]
第6行预测完成！
[267.]
第7行预测完成！
[407.]
第8行预测完成！
[538.]
第9行预测完成！
[465.]
第10行预测完成！
[654.]
第11行预测完成！
[757.]
第12行预测完成！
[664.]
第13行预测完成！
[457.]
第14行预测完成！
[406.]
第15行预测完成！
[407.]
第16行预测完成！
[628.]
第17行预测完成！
[646.]
第18行预测完成！
[499.]
第19行预测完成！
[434.]
第20行预测完成！
[441.]
第21行预测完成！
[475.]
第22行预测完成！
[272.]
第23行预测完成！
[166.]
第24行预测完成！
[126.]
第25行预测完成！
[115.]
第26行预测完成！
[106.]
第27行预测完成！
[111.]
第28行预测完成！
[117.]
第29行预测完成！
[172.]
第30行预测完成！
[468.]
第31行预测完成！
[691.]
第32行预测完成！
[671.]
第33行预测完成！
[534.]
第34行预测完成！
[785.]
第35行预测完成！
[987.]
第36行预测完成！
[766.]
第37行预测完成！
[581.]
第38行预测完成！
[522.]
第39行预测完成！
[513.]
第40行预测完成！
[798.]
第41行预测完成！
[791.]
第42行预测完成！
[560.]
第43行预测完成！
[524.]
第44行预测完成！
[530.]
第45行预测完成！
[529.]
第46行预测完成！
[332.]
第47行预测完成！
[176.]
第48行预测完成！
[131.]
第49行预测完成！
[111.]
第50行预测完成！
[111.]
第51行预测完成！
[115.]
第52行预测完成！
[113.]
第53行预测完成！
[179.]
第54行预测完成！
[552.]
第55行预测完成！
[721.]
第56行预测完成！
[663.]
第57行预测完成！
[529.]
第58行预测完成！
[812.]
第59行

[692.]
第464行预测完成！
[615.]
第465行预测完成！
[548.]
第466行预测完成！
[918.]
第467行预测完成！
[939.]
第468行预测完成！
[703.]
第469行预测完成！
[585.]
第470行预测完成！
[537.]
第471行预测完成！
[597.]
第472行预测完成！
[834.]
第473行预测完成！
[751.]
第474行预测完成！
[538.]
第475行预测完成！
[469.]
第476行预测完成！
[460.]
第477行预测完成！
[432.]
第478行预测完成！
[221.]
第479行预测完成！
[119.]
第480行预测完成！
[92.]
第481行预测完成！
[83.]
第482行预测完成！
[79.]
第483行预测完成！
[83.]
第484行预测完成！
[110.]
第485行预测完成！
[191.]
第486行预测完成！
[337.]
第487行预测完成！
[455.]
第488行预测完成！
[498.]
第489行预测完成！
[496.]
第490行预测完成！
[767.]
第491行预测完成！
[764.]
第492行预测完成！
[552.]
第493行预测完成！
[473.]
第494行预测完成！
[436.]
第495行预测完成！
[507.]
第496行预测完成！
[674.]
第497行预测完成！
[658.]
第498行预测完成！
[493.]
第499行预测完成！
[416.]
第500行预测完成！
[399.]
第501行预测完成！
[391.]
第502行预测完成！
[221.]
第503行预测完成！
[114.]
第504行预测完成！
[88.]
第505行预测完成！
[77.]
第506行预测完成！
[70.]
第507行预测完成！
[74.]
第508行预测完成！
[103.]
第509行预测完成！
[170.]
第510行预测完成！
[308.]
第511行预测完成！
[404.]
第512行预测完成！
[468.]
第513行预测完成！
[501.]
第514行预测完成！
[683.]
第515行预测完成！
[652.]
第516行预测完成！
[533.]
第517行预测完成！
[447.]
第518行预测完成！
[422.]
第519行预测完成！


[645.]
第181行预测完成！
[501.]
第182行预测完成！
[462.]
第183行预测完成！
[452.]
第184行预测完成！
[683.]
第185行预测完成！
[697.]
第186行预测完成！
[504.]
第187行预测完成！
[403.]
第188行预测完成！
[405.]
第189行预测完成！
[487.]
第190行预测完成！
[318.]
第191行预测完成！
[254.]
第192行预测完成！
[229.]
第193行预测完成！
[233.]
第194行预测完成！
[218.]
第195行预测完成！
[221.]
第196行预测完成！
[218.]
第197行预测完成！
[244.]
第198行预测完成！
[576.]
第199行预测完成！
[669.]
第200行预测完成！
[627.]
第201行预测完成！
[529.]
第202行预测完成！
[835.]
第203行预测完成！
[1042.]
第204行预测完成！
[890.]
第205行预测完成！
[694.]
第206行预测完成！
[619.]
第207行预测完成！
[559.]
第208行预测完成！
[909.]
第209行预测完成！
[893.]
第210行预测完成！
[663.]
第211行预测完成！
[542.]
第212行预测完成！
[549.]
第213行预测完成！
[594.]
第214行预测完成！
[391.]
第215行预测完成！
[242.]
第216行预测完成！
[221.]
第217行预测完成！
[219.]
第218行预测完成！
[212.]
第219行预测完成！
[215.]
第220行预测完成！
[210.]
第221行预测完成！
[247.]
第222行预测完成！
[580.]
第223行预测完成！
[679.]
第224行预测完成！
[643.]
第225行预测完成！
[540.]
第226行预测完成！
[854.]
第227行预测完成！
[1101.]
第228行预测完成！
[930.]
第229行预测完成！
[683.]
第230行预测完成！
[625.]
第231行预测完成！
[545.]
第232行预测完成！
[943.]
第233行预测完成！
[906.]
第234行预测完成！
[661.]
第235行预测完成！
[545.]
第

[811.]
第637行预测完成！
[687.]
第638行预测完成！
[639.]
第639行预测完成！
[591.]
第640行预测完成！
[882.]
第641行预测完成！
[865.]
第642行预测完成！
[662.]
第643行预测完成！
[590.]
第644行预测完成！
[576.]
第645行预测完成！
[570.]
第646行预测完成！
[360.]
第647行预测完成！
[212.]
第648行预测完成！
[184.]
第649行预测完成！
[168.]
第650行预测完成！
[172.]
第651行预测完成！
[173.]
第652行预测完成！
[177.]
第653行预测完成！
[214.]
第654行预测完成！
[370.]
第655行预测完成！
[523.]
第656行预测完成！
[526.]
第657行预测完成！
[503.]
第658行预测完成！
[758.]
第659行预测完成！
[838.]
第660行预测完成！
[654.]
第661行预测完成！
[546.]
第662行预测完成！
[496.]
第663行预测完成！
[504.]
第664行预测完成！
[694.]
第665行预测完成！
[714.]
第666行预测完成！
[558.]
第667行预测完成！
[460.]
第668行预测完成！
[441.]
第669行预测完成！
[451.]
第670行预测完成！
[319.]
第671行预测完成！
[198.]
第672行预测完成！
[169.]
第673行预测完成！
[154.]
第674行预测完成！
[147.]
第675行预测完成！
[148.]
第676行预测完成！
[157.]
第677行预测完成！
[206.]
第678行预测完成！
[357.]
第679行预测完成！
[483.]
第680行预测完成！
[483.]
第681行预测完成！
[477.]
第682行预测完成！
[665.]
第683行预测完成！
[763.]
第684行预测完成！
[605.]
第685行预测完成！
[513.]
第686行预测完成！
[469.]
第687行预测完成！
[485.]
第688行预测完成！
[641.]
第689行预测完成！
[642.]
第690行预测完成！
[523.]
第691行预测完成！
[431.]
第69

...

第273行预测完成！
[1057.]
第274行预测完成！
[1873.]
第275行预测完成！
[2304.]
第276行预测完成！
[1596.]
第277行预测完成！
[1241.]
第278行预测完成！
[1174.]
第279行预测完成！
[1183.]
第280行预测完成！
[1944.]
第281行预测完成！
[1975.]
第282行预测完成！
[1278.]
第283行预测完成！
[1121.]
第284行预测完成！
[1089.]
第285行预测完成！
[971.]
第286行预测完成！
[373.]
第287行预测完成！
[74.]
第288行预测完成！
[23.]
第289行预测完成！
[17.]
第290行预测完成！
[15.]
第291行预测完成！
[19.]
第292行预测完成！
[65.]
第293行预测完成！
[246.]
第294行预测完成！
[1182.]
第295行预测完成！
[1383.]
第296行预测完成！
[1140.]
第297行预测完成！
[962.]
第298行预测完成！
[1721.]
第299行预测完成！
[2258.]
第300行预测完成！
[1538.]
第301行预测完成！
[1093.]
第302行预测完成！
[1070.]
第303行预测完成！
[1018.]
第304行预测完成！
[1758.]
第305行预测完成！
[1622.]
第306行预测完成！
[1172.]
第307行预测完成！
[992.]
第308行预测完成！
[969.]
第309行预测完成！
[847.]
第310行预测完成！
[325.]
第311行预测完成！
[69.]
第312行预测完成！
[18.]
第313行预测完成！
[10.]
第314行预测完成！
[9.]
第315行预测完成！
[13.]
第316行预测完成！
[46.]
第317行预测完成！
[158.]
第318行预测完成！
[605.]
第319行预测完成！
[871.]
第320行预测完成！
[863.]
第321行预测完成！
[851.]
第322行预测完成！
[1424.]
第323行预测完成！
[1661.]
第324行预测完成！
[1161.]
第325行预测完成！
[849.]
第326行预测完成！
[782.]
第327行预测完成！
[85

第726行预测完成！
[1244.]
第727行预测完成！
[1648.]
第728行预测完成！
[1263.]
第729行预测完成！
[999.]
第730行预测完成！
[1819.]
第731行预测完成！
[2416.]
第732行预测完成！
[1563.]
第733行预测完成！
[1170.]
第734行预测完成！
[1197.]
第735行预测完成！
[1202.]
第736行预测完成！
[1955.]
第737行预测完成！
[1837.]
第738行预测完成！
[1289.]
第739行预测完成！
[1101.]
第740行预测完成！
[1023.]
第741行预测完成！
[969.]
第742行预测完成！
[350.]
第743行预测完成！
******删减前地点数据大小：1464******
******删减后地点数据大小：744******
[58.]
第0行预测完成！
[45.]
第1行预测完成！
[44.]
第2行预测完成！
[46.]
第3行预测完成！
[43.]
第4行预测完成！
[45.]
第5行预测完成！
[80.]
第6行预测完成！
[184.]
第7行预测完成！
[298.]
第8行预测完成！
[375.]
第9行预测完成！
[353.]
第10行预测完成！
[567.]
第11行预测完成！
[697.]
第12行预测完成！
[464.]
第13行预测完成！
[349.]
第14行预测完成！
[341.]
第15行预测完成！
[344.]
第16行预测完成！
[521.]
第17行预测完成！
[585.]
第18行预测完成！
[426.]
第19行预测完成！
[353.]
第20行预测完成！
[360.]
第21行预测完成！
[377.]
第22行预测完成！
[212.]
第23行预测完成！
[112.]
第24行预测完成！
[98.]
第25行预测完成！
[97.]
第26行预测完成！
[98.]
第27行预测完成！
[98.]
第28行预测完成！
[105.]
第29行预测完成！
[136.]
第30行预测完成！
[354.]
第31行预测完成！
[565.]
第32行预测完成！
[467.]
第33行预测完成！
[385.]
第34行预测完成！
[613.]
第35行预测完成！
[865.]
第36行预测完成！
[590.]
第

第447行预测完成！
[494.]
第448行预测完成！
[868.]
第449行预测完成！
[773.]
第450行预测完成！
[510.]
第451行预测完成！
[433.]
第452行预测完成！
[453.]
第453行预测完成！
[448.]
第454行预测完成！
[209.]
第455行预测完成！
[87.]
第456行预测完成！
[70.]
第457行预测完成！
[65.]
第458行预测完成！
[60.]
第459行预测完成！
[56.]
第460行预测完成！
[68.]
第461行预测完成！
[181.]
第462行预测完成！
[522.]
第463行预测完成！
[615.]
第464行预测完成！
[476.]
第465行预测完成！
[396.]
第466行预测完成！
[735.]
第467行预测完成！
[824.]
第468行预测完成！
[567.]
第469行预测完成！
[474.]
第470行预测完成！
[424.]
第471行预测完成！
[452.]
第472行预测完成！
[787.]
第473行预测完成！
[693.]
第474行预测完成！
[460.]
第475行预测完成！
[390.]
第476行预测完成！
[409.]
第477行预测完成！
[409.]
第478行预测完成！
[192.]
第479行预测完成！
[80.]
第480行预测完成！
[62.]
第481行预测完成！
[47.]
第482行预测完成！
[42.]
第483行预测完成！
[38.]
第484行预测完成！
[56.]
第485行预测完成！
[140.]
第486行预测完成！
[292.]
第487行预测完成！
[383.]
第488行预测完成！
[390.]
第489行预测完成！
[372.]
第490行预测完成！
[629.]
第491行预测完成！
[670.]
第492行预测完成！
[453.]
第493行预测完成！
[365.]
第494行预测完成！
[354.]
第495行预测完成！
[392.]
第496行预测完成！
[555.]
第497行预测完成！
[553.]
第498行预测完成！
[420.]
第499行预测完成！
[360.]
第500行预测完成！
[360.]
第501行预测完成！
[367.]
第502行预测完成！
[187.]
第503

第172行预测完成！
[3.]
第173行预测完成！
[21.]
第174行预测完成！
[126.]
第175行预测完成！
[242.]
第176行预测完成！
[340.]
第177行预测完成！
[408.]
第178行预测完成！
[442.]
第179行预测完成！
[434.]
第180行预测完成！
[379.]
第181行预测完成！
[353.]
第182行预测完成！
[387.]
第183行预测完成！
[449.]
第184行预测完成！
[544.]
第185行预测完成！
[491.]
第186行预测完成！
[387.]
第187行预测完成！
[343.]
第188行预测完成！
[302.]
第189行预测完成！
[221.]
第190行预测完成！
[57.]
第191行预测完成！
[14.]
第192行预测完成！
[9.]
第193行预测完成！
[6.]
第194行预测完成！
[8.]
第195行预测完成！
[12.]
第196行预测完成！
[23.]
第197行预测完成！
[81.]
第198行预测完成！
[387.]
第199行预测完成！
[786.]
第200行预测完成！
[865.]
第201行预测完成！
[785.]
第202行预测完成！
[928.]
第203行预测完成！
[836.]
第204行预测完成！
[789.]
第205行预测完成！
[778.]
第206行预测完成！
[808.]
第207行预测完成！
[825.]
第208行预测完成！
[1087.]
第209行预测完成！
[878.]
第210行预测完成！
[573.]
第211行预测完成！
[532.]
第212行预测完成！
[457.]
第213行预测完成！
[266.]
第214行预测完成！
[59.]
第215行预测完成！
[14.]
第216行预测完成！
[8.]
第217行预测完成！
[9.]
第218行预测完成！
[10.]
第219行预测完成！
[12.]
第220行预测完成！
[31.]
第221行预测完成！
[111.]
第222行预测完成！
[537.]
第223行预测完成！
[752.]
第224行预测完成！
[838.]
第225行预测完成！
[758.]
第226行预测完成！
[909.]
第227行预测完成！
[771.]
第228行预测完成！
[70

第639行预测完成！
[846.]
第640行预测完成！
[819.]
第641行预测完成！
[604.]
第642行预测完成！
[442.]
第643行预测完成！
[416.]
第644行预测完成！
[364.]
第645行预测完成！
[171.]
第646行预测完成！
[40.]
第647行预测完成！
[7.]
第648行预测完成！
[1.]
第649行预测完成！
[-1.]
第650行预测完成！
[-1.]
第651行预测完成！
[2.]
第652行预测完成！
[13.]
第653行预测完成！
[75.]
第654行预测完成！
[274.]
第655行预测完成！
[364.]
第656行预测完成！
[394.]
第657行预测完成！
[452.]
第658行预测完成！
[492.]
第659行预测完成！
[435.]
第660行预测完成！
[367.]
第661行预测完成！
[323.]
第662行预测完成！
[280.]
第663行预测完成！
[327.]
第664行预测完成！
[412.]
第665行预测完成！
[389.]
第666行预测完成！
[333.]
第667行预测完成！
[318.]
第668行预测完成！
[304.]
第669行预测完成！
[205.]
第670行预测完成！
[60.]
第671行预测完成！
[8.]
第672行预测完成！
[0.]
第673行预测完成！
[-2.]
第674行预测完成！
[-1.]
第675行预测完成！
[-1.]
第676行预测完成！
[4.]
第677行预测完成！
[29.]
第678行预测完成！
[136.]
第679行预测完成！
[266.]
第680行预测完成！
[306.]
第681行预测完成！
[359.]
第682行预测完成！
[413.]
第683行预测完成！
[430.]
第684行预测完成！
[345.]
第685行预测完成！
[339.]
第686行预测完成！
[331.]
第687行预测完成！
[352.]
第688行预测完成！
[449.]
第689行预测完成！
[442.]
第690行预测完成！
[365.]
第691行预测完成！
[333.]
第692行预测完成！
[301.]
第693行预测完成！
[214.]
第694行预测完成！
[66.]
第695行预测完成！
[16.

In [39]:
test_df

""


In [147]:
pred = test_df.num_of_people
real = test_y
rmse = np.sqrt(mean_squared_error(pred,real))
print('Test RMSE:',rmse)

Test RMSE: 925.1023492291891


## 4. 11月份预测导出

In [15]:
nov_data = pd.read_csv('../datasets/Nov_test_data.csv')

In [17]:
%time nov_data = real_time_predict(model_xgb,nov_data)

C:\Tool\Anaconda\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


[ 543.]
第0行预测完成！
[ 439.]
第1行预测完成！
[ 332.]
第2行预测完成！
[ 309.]
第3行预测完成！
[ 300.]
第4行预测完成！
[ 315.]
第5行预测完成！
[ 308.]
第6行预测完成！
[ 543.]
第7行预测完成！
[ 662.]
第8行预测完成！
[ 641.]
第9行预测完成！
[ 567.]
第10行预测完成！
[ 908.]
第11行预测完成！
[ 1089.]
第12行预测完成！
[ 859.]
第13行预测完成！
[ 575.]
第14行预测完成！
[ 550.]
第15行预测完成！
[ 624.]
第16行预测完成！
[ 871.]
第17行预测完成！
[ 903.]
第18行预测完成！
[ 661.]
第19行预测完成！
[ 636.]
第20行预测完成！
[ 633.]
第21行预测完成！
[ 668.]
第22行预测完成！
[ 462.]
第23行预测完成！
[ 506.]
第24行预测完成！
[ 393.]
第25行预测完成！
[ 323.]
第26行预测完成！
[ 313.]
第27行预测完成！
[ 290.]
第28行预测完成！
[ 304.]
第29行预测完成！
[ 298.]
第30行预测完成！
[ 546.]
第31行预测完成！
[ 682.]
第32行预测完成！
[ 655.]
第33行预测完成！
[ 594.]
第34行预测完成！
[ 866.]
第35行预测完成！
[ 1150.]
第36行预测完成！
[ 918.]
第37行预测完成！
[ 629.]
第38行预测完成！
[ 621.]
第39行预测完成！
[ 605.]
第40行预测完成！
[ 888.]
第41行预测完成！
[ 924.]
第42行预测完成！
[ 655.]
第43行预测完成！
[ 573.]
第44行预测完成！
[ 613.]
第45行预测完成！
[ 608.]
第46行预测完成！
[ 423.]
第47行预测完成！
[ 425.]
第48行预测完成！
[ 374.]
第49行预测完成！
[ 318.]
第50行预测完成！
[ 304.]
第51行预测完成！
[ 296.]
第52行预测完成！
[ 309.]
第53行预测完成！
[ 296.]
第54行预测完成！
[ 489.]
第55行预测完成！


第438行预测完成！
[ 330.]
第439行预测完成！
[ 428.]
第440行预测完成！
[ 516.]
第441行预测完成！
[ 500.]
第442行预测完成！
[ 856.]
第443行预测完成！
[ 1061.]
第444行预测完成！
[ 835.]
第445行预测完成！
[ 617.]
第446行预测完成！
[ 566.]
第447行预测完成！
[ 608.]
第448行预测完成！
[ 905.]
第449行预测完成！
[ 958.]
第450行预测完成！
[ 698.]
第451行预测完成！
[ 461.]
第452行预测完成！
[ 517.]
第453行预测完成！
[ 539.]
第454行预测完成！
[ 349.]
第455行预测完成！
[ 395.]
第456行预测完成！
[ 378.]
第457行预测完成！
[ 331.]
第458行预测完成！
[ 370.]
第459行预测完成！
[ 331.]
第460行预测完成！
[ 323.]
第461行预测完成！
[ 309.]
第462行预测完成！
[ 473.]
第463行预测完成！
[ 611.]
第464行预测完成！
[ 632.]
第465行预测完成！
[ 626.]
第466行预测完成！
[ 984.]
第467行预测完成！
[ 1190.]
第468行预测完成！
[ 947.]
第469行预测完成！
[ 710.]
第470行预测完成！
[ 677.]
第471行预测完成！
[ 732.]
第472行预测完成！
[ 1026.]
第473行预测完成！
[ 1058.]
第474行预测完成！
[ 737.]
第475行预测完成！
[ 629.]
第476行预测完成！
[ 694.]
第477行预测完成！
[ 711.]
第478行预测完成！
[ 425.]
第479行预测完成！
[ 410.]
第480行预测完成！
[ 369.]
第481行预测完成！
[ 318.]
第482行预测完成！
[ 348.]
第483行预测完成！
[ 314.]
第484行预测完成！
[ 310.]
第485行预测完成！
[ 313.]
第486行预测完成！
[ 591.]
第487行预测完成！
[ 749.]
第488行预测完成！
[ 721.]
第489行预测完成！
[ 625.]
第490行预测完

第155行预测完成！
[ 1176.]
第156行预测完成！
[ 915.]
第157行预测完成！
[ 543.]
第158行预测完成！
[ 680.]
第159行预测完成！
[ 518.]
第160行预测完成！
[ 1077.]
第161行预测完成！
[ 961.]
第162行预测完成！
[ 584.]
第163行预测完成！
[ 546.]
第164行预测完成！
[ 627.]
第165行预测完成！
[ 790.]
第166行预测完成！
[ 785.]
第167行预测完成！
[ 771.]
第168行预测完成！
[ 960.]
第169行预测完成！
[ 1077.]
第170行预测完成！
[ 1122.]
第171行预测完成！
[ 1178.]
第172行预测完成！
[ 1255.]
第173行预测完成！
[ 1093.]
第174行预测完成！
[ 980.]
第175行预测完成！
[ 600.]
第176行预测完成！
[ 536.]
第177行预测完成！
[ 427.]
第178行预测完成！
[ 764.]
第179行预测完成！
[ 1176.]
第180行预测完成！
[ 927.]
第181行预测完成！
[ 570.]
第182行预测完成！
[ 607.]
第183行预测完成！
[ 497.]
第184行预测完成！
[ 1026.]
第185行预测完成！
[ 941.]
第186行预测完成！
[ 583.]
第187行预测完成！
[ 537.]
第188行预测完成！
[ 572.]
第189行预测完成！
[ 750.]
第190行预测完成！
[ 737.]
第191行预测完成！
[ 676.]
第192行预测完成！
[ 874.]
第193行预测完成！
[ 936.]
第194行预测完成！
[ 1027.]
第195行预测完成！
[ 1127.]
第196行预测完成！
[ 1181.]
第197行预测完成！
[ 969.]
第198行预测完成！
[ 874.]
第199行预测完成！
[ 577.]
第200行预测完成！
[ 551.]
第201行预测完成！
[ 421.]
第202行预测完成！
[ 773.]
第203行预测完成！
[ 1195.]
第204行预测完成！
[ 891.]
第205行预测完成！
[ 566.]
第206行预测完成！
[ 653.]

第585行预测完成！
[ 354.]
第586行预测完成！
[ 714.]
第587行预测完成！
[ 993.]
第588行预测完成！
[ 869.]
第589行预测完成！
[ 547.]
第590行预测完成！
[ 474.]
第591行预测完成！
[ 449.]
第592行预测完成！
[ 873.]
第593行预测完成！
[ 874.]
第594行预测完成！
[ 517.]
第595行预测完成！
[ 403.]
第596行预测完成！
[ 442.]
第597行预测完成！
[ 666.]
第598行预测完成！
[ 610.]
第599行预测完成！
[ 663.]
第600行预测完成！
[ 783.]
第601行预测完成！
[ 929.]
第602行预测完成！
[ 1080.]
第603行预测完成！
[ 1153.]
第604行预测完成！
[ 1187.]
第605行预测完成！
[ 1102.]
第606行预测完成！
[ 956.]
第607行预测完成！
[ 619.]
第608行预测完成！
[ 502.]
第609行预测完成！
[ 359.]
第610行预测完成！
[ 754.]
第611行预测完成！
[ 989.]
第612行预测完成！
[ 859.]
第613行预测完成！
[ 496.]
第614行预测完成！
[ 408.]
第615行预测完成！
[ 461.]
第616行预测完成！
[ 883.]
第617行预测完成！
[ 863.]
第618行预测完成！
[ 543.]
第619行预测完成！
[ 350.]
第620行预测完成！
[ 428.]
第621行预测完成！
[ 667.]
第622行预测完成！
[ 669.]
第623行预测完成！
[ 709.]
第624行预测完成！
[ 853.]
第625行预测完成！
[ 1060.]
第626行预测完成！
[ 1248.]
第627行预测完成！
[ 1284.]
第628行预测完成！
[ 1255.]
第629行预测完成！
[ 1195.]
第630行预测完成！
[ 1131.]
第631行预测完成！
[ 764.]
第632行预测完成！
[ 614.]
第633行预测完成！
[ 433.]
第634行预测完成！
[ 792.]
第635行预测完成！
[ 1159.]
第636行预测完成！
[ 980.]
第

...

第124行预测完成！
[ 80.]
第125行预测完成！
[ 171.]
第126行预测完成！
[ 837.]
第127行预测完成！
[ 1225.]
第128行预测完成！
[ 1144.]
第129行预测完成！
[ 834.]
第130行预测完成！
[ 1681.]
第131行预测完成！
[ 2179.]
第132行预测完成！
[ 1510.]
第133行预测完成！
[ 1115.]
第134行预测完成！
[ 1147.]
第135行预测完成！
[ 872.]
第136行预测完成！
[ 1919.]
第137行预测完成！
[ 1891.]
第138行预测完成！
[ 1249.]
第139行预测完成！
[ 1135.]
第140行预测完成！
[ 1130.]
第141行预测完成！
[ 1277.]
第142行预测完成！
[ 533.]
第143行预测完成！
[ 175.]
第144行预测完成！
[ 130.]
第145行预测完成！
[ 111.]
第146行预测完成！
[ 100.]
第147行预测完成！
[ 97.]
第148行预测完成！
[ 100.]
第149行预测完成！
[ 195.]
第150行预测完成！
[ 1193.]
第151行预测完成！
[ 1364.]
第152行预测完成！
[ 1177.]
第153行预测完成！
[ 882.]
第154行预测完成！
[ 1779.]
第155行预测完成！
[ 2568.]
第156行预测完成！
[ 1664.]
第157行预测完成！
[ 1177.]
第158行预测完成！
[ 1188.]
第159行预测完成！
[ 951.]
第160行预测完成！
[ 2049.]
第161行预测完成！
[ 2075.]
第162行预测完成！
[ 1374.]
第163行预测完成！
[ 1165.]
第164行预测完成！
[ 1208.]
第165行预测完成！
[ 1353.]
第166行预测完成！
[ 463.]
第167行预测完成！
[ 172.]
第168行预测完成！
[ 130.]
第169行预测完成！
[ 117.]
第170行预测完成！
[ 107.]
第171行预测完成！
[ 102.]
第172行预测完成！
[ 110.]
第173行预测完成！
[ 220.]
第174行预测完成！
[ 1182.]
第175行

第548行预测完成！
[ 1226.]
第549行预测完成！
[ 1355.]
第550行预测完成！
[ 493.]
第551行预测完成！
[ 155.]
第552行预测完成！
[ 117.]
第553行预测完成！
[ 101.]
第554行预测完成！
[ 103.]
第555行预测完成！
[ 102.]
第556行预测完成！
[ 118.]
第557行预测完成！
[ 247.]
第558行预测完成！
[ 1152.]
第559行预测完成！
[ 1468.]
第560行预测完成！
[ 1274.]
第561行预测完成！
[ 992.]
第562行预测完成！
[ 1859.]
第563行预测完成！
[ 2794.]
第564行预测完成！
[ 1735.]
第565行预测完成！
[ 1208.]
第566行预测完成！
[ 1260.]
第567行预测完成！
[ 1018.]
第568行预测完成！
[ 2012.]
第569行预测完成！
[ 1915.]
第570行预测完成！
[ 1326.]
第571行预测完成！
[ 1023.]
第572行预测完成！
[ 1091.]
第573行预测完成！
[ 1242.]
第574行预测完成！
[ 443.]
第575行预测完成！
[ 130.]
第576行预测完成！
[ 104.]
第577行预测完成！
[ 94.]
第578行预测完成！
[ 101.]
第579行预测完成！
[ 100.]
第580行预测完成！
[ 112.]
第581行预测完成！
[ 216.]
第582行预测完成！
[ 756.]
第583行预测完成！
[ 1175.]
第584行预测完成！
[ 1096.]
第585行预测完成！
[ 953.]
第586行预测完成！
[ 1759.]
第587行预测完成！
[ 2517.]
第588行预测完成！
[ 1553.]
第589行预测完成！
[ 1059.]
第590行预测完成！
[ 1076.]
第591行预测完成！
[ 919.]
第592行预测完成！
[ 1657.]
第593行预测完成！
[ 1742.]
第594行预测完成！
[ 1173.]
第595行预测完成！
[ 801.]
第596行预测完成！
[ 807.]
第597行预测完成！
[ 979.]
第598行预测完成！
[ 370.]
第599行

第266行预测完成！
[ 8.]
第267行预测完成！
[ 11.]
第268行预测完成！
[ 8.]
第269行预测完成！
[ 37.]
第270行预测完成！
[ 128.]
第271行预测完成！
[ 246.]
第272行预测完成！
[ 338.]
第273行预测完成！
[ 271.]
第274行预测完成！
[ 645.]
第275行预测完成！
[ 845.]
第276行预测完成！
[ 491.]
第277行预测完成！
[ 356.]
第278行预测完成！
[ 320.]
第279行预测完成！
[ 318.]
第280行预测完成！
[ 706.]
第281行预测完成！
[ 693.]
第282行预测完成！
[ 484.]
第283行预测完成！
[ 314.]
第284行预测完成！
[ 337.]
第285行预测完成！
[ 413.]
第286行预测完成！
[ 210.]
第287行预测完成！
[ 46.]
第288行预测完成！
[ 33.]
第289行预测完成！
[ 19.]
第290行预测完成！
[ 26.]
第291行预测完成！
[ 28.]
第292行预测完成！
[ 34.]
第293行预测完成！
[ 57.]
第294行预测完成！
[ 249.]
第295行预测完成！
[ 567.]
第296行预测完成！
[ 386.]
第297行预测完成！
[ 253.]
第298行预测完成！
[ 701.]
第299行预测完成！
[ 1046.]
第300行预测完成！
[ 666.]
第301行预测完成！
[ 482.]
第302行预测完成！
[ 397.]
第303行预测完成！
[ 336.]
第304行预测完成！
[ 911.]
第305行预测完成！
[ 895.]
第306行预测完成！
[ 562.]
第307行预测完成！
[ 409.]
第308行预测完成！
[ 437.]
第309行预测完成！
[ 543.]
第310行预测完成！
[ 244.]
第311行预测完成！
[ 59.]
第312行预测完成！
[ 26.]
第313行预测完成！
[ 20.]
第314行预测完成！
[ 21.]
第315行预测完成！
[ 27.]
第316行预测完成！
[ 20.]
第317行预测完成！
[ 61.]
第318行预测完成！
[ 321.]
第319行预测完成！
[

第708行预测完成！
[ 669.]
第709行预测完成！
[ 514.]
第710行预测完成！
[ 436.]
第711行预测完成！
[ 354.]
第712行预测完成！
[ 905.]
第713行预测完成！
[ 878.]
第714行预测完成！
[ 560.]
第715行预测完成！
[ 423.]
第716行预测完成！
[ 455.]
第717行预测完成！
[ 445.]
第718行预测完成！
[ 177.]
第719行预测完成！
******删减前地点数据大小：1440******
******删减后地点数据大小：720******
[ 41.]
第0行预测完成！
[ 21.]
第1行预测完成！
[ 20.]
第2行预测完成！
[ 22.]
第3行预测完成！
[ 20.]
第4行预测完成！
[ 26.]
第5行预测完成！
[ 54.]
第6行预测完成！
[ 280.]
第7行预测完成！
[ 433.]
第8行预测完成！
[ 652.]
第9行预测完成！
[ 662.]
第10行预测完成！
[ 821.]
第11行预测完成！
[ 754.]
第12行预测完成！
[ 641.]
第13行预测完成！
[ 591.]
第14行预测完成！
[ 666.]
第15行预测完成！
[ 621.]
第16行预测完成！
[ 825.]
第17行预测完成！
[ 634.]
第18行预测完成！
[ 404.]
第19行预测完成！
[ 419.]
第20行预测完成！
[ 362.]
第21行预测完成！
[ 256.]
第22行预测完成！
[ 60.]
第23行预测完成！
[ 17.]
第24行预测完成！
[ 13.]
第25行预测完成！
[ 14.]
第26行预测完成！
[ 14.]
第27行预测完成！
[ 16.]
第28行预测完成！
[ 24.]
第29行预测完成！
[ 62.]
第30行预测完成！
[ 330.]
第31行预测完成！
[ 548.]
第32行预测完成！
[ 836.]
第33行预测完成！
[ 753.]
第34行预测完成！
[ 864.]
第35行预测完成！
[ 660.]
第36行预测完成！
[ 736.]
第37行预测完成！
[ 744.]
第38行预测完成！
[ 920.]
第39行预测完成！
[ 709.]
第40行预测完成！
[ 963.]
第41行预测

第432行预测完成！
[ 15.]
第433行预测完成！
[ 9.]
第434行预测完成！
[ 12.]
第435行预测完成！
[ 13.]
第436行预测完成！
[ 19.]
第437行预测完成！
[ 54.]
第438行预测完成！
[ 146.]
第439行预测完成！
[ 241.]
第440行预测完成！
[ 292.]
第441行预测完成！
[ 278.]
第442行预测完成！
[ 320.]
第443行预测完成！
[ 278.]
第444行预测完成！
[ 234.]
第445行预测完成！
[ 239.]
第446行预测完成！
[ 260.]
第447行预测完成！
[ 305.]
第448行预测完成！
[ 358.]
第449行预测完成！
[ 287.]
第450行预测完成！
[ 222.]
第451行预测完成！
[ 195.]
第452行预测完成！
[ 170.]
第453行预测完成！
[ 118.]
第454行预测完成！
[ 22.]
第455行预测完成！
[ 6.]
第456行预测完成！
[ 12.]
第457行预测完成！
[ 10.]
第458行预测完成！
[ 11.]
第459行预测完成！
[ 12.]
第460行预测完成！
[ 20.]
第461行预测完成！
[ 56.]
第462行预测完成！
[ 222.]
第463行预测完成！
[ 485.]
第464行预测完成！
[ 811.]
第465行预测完成！
[ 776.]
第466行预测完成！
[ 855.]
第467行预测完成！
[ 766.]
第468行预测完成！
[ 744.]
第469行预测完成！
[ 752.]
第470行预测完成！
[ 945.]
第471行预测完成！
[ 771.]
第472行预测完成！
[ 1123.]
第473行预测完成！
[ 688.]
第474行预测完成！
[ 457.]
第475行预测完成！
[ 461.]
第476行预测完成！
[ 380.]
第477行预测完成！
[ 271.]
第478行预测完成！
[ 53.]
第479行预测完成！
[ 18.]
第480行预测完成！
[ 13.]
第481行预测完成！
[ 11.]
第482行预测完成！
[ 12.]
第483行预测完成！
[ 12.]
第484行预测完成！
[ 17.]
第485行预测完成！
[ 49

In [18]:
nov_data.sort_values(by=['time_stamp','loc_id'],inplace=True)

In [20]:
nov_data.iloc[:,:3].to_csv('../datasets/nov_test.csv',index=False)

# 公共函数

In [10]:
# 按地点分组
def locID_split(data):
    res = []
    df = data.copy()
    for i in df.loc_id.unique():
        tmp = df[df.loc_id == i]
        tmp.reset_index(inplace=True)
        tmp.drop('index',inplace=True,axis=1)
        res.append(tmp)
    return res

In [11]:
# 实时预测填充
def real_time_predict(model,data):
    df = data.copy()
    test_list = locID_split(df)
    res = pd.DataFrame()
    for tmp in test_list:
        for i in range(tmp.shape[0]):
            X = xgb.DMatrix(tmp.iloc[i:i+1,3:])
            pred = np.round(model.predict(X,ntree_limit = model.best_iteration))
            print(pred)
            tmp.loc[i,'num_of_people'] = pred
            for j in range(len(columns_24h)):
                tmp.loc[i+j+1,columns_24h[j]] = pred
            for k in range(len(columns_30d)):
                tmp.loc[i+(k+1)*24,columns_30d[k]] = pred
            print('第%s行预测完成！'%i)
        print('******删减前地点数据大小：%s******'%tmp.shape[0])
        tmp_notnull = tmp[tmp.time_stamp.notnull()]
        print('******删减后地点数据大小：%s******'%tmp_notnull.shape[0])
        res = res.append(tmp_notnull)
    return res

In [12]:
reg = re.compile('\s+|\t|\n') # 正则匹配空格、Tab键、换行
#列名，历史24小时人数
col_24h='''history_1h_population,history_2h_population,history_3h_population,history_4h_population
    ,history_5h_population,history_6h_population,history_7h_population,history_8h_population
    ,history_9h_population,history_10h_population,history_11h_population,history_12h_population
    ,history_13h_population,history_14h_population,history_15h_population,history_16h_population,history_17h_population
    ,history_18h_population,history_19h_population,history_20h_population,history_21h_population,history_22h_population
    ,history_23h_population'''
columns_24h=re.sub(reg,'',col_24h).split(',')  
#列名，同一时刻历史15天人数
col_30d='''history_1d_population,history_2d_population,history_3d_population,
    history_4d_population,history_5d_population,history_6d_population,
    history_7d_population,history_8d_population,history_9d_population,history_10d_population,history_11d_population,
    history_12d_population,history_13d_population,history_14d_population,history_15d_population,history_16d_population
    ,history_17d_population,history_18d_population,history_19d_population,history_20d_population,history_21d_population
    ,history_22d_population,history_23d_population,history_24d_population,history_25d_population,history_26d_population
    ,history_27d_population,history_28d_population,history_29d_population,history_30d_population'''
columns_30d=re.sub(reg,'',col_30d).split(',')  

In [13]:
def xgboost_model_train(train_X,train_y,test_X,test_y):
    # 模型训练
    xgb_train  = xgb.DMatrix(train_X, train_y)
    xgb_test   = xgb.DMatrix(test_X, test_y)
    params_xgb = {
#         'booster': 'gbtree',                # 取决于使用哪种booster,有两中模型可以选择gbtree和gblinear。gbtree使用基于树的模型进行提升计算，gblinear使用线性模型进行提升计算
        'objective': 'reg:linear',     # 定义学习任务及相应的学习目标，可选的目标函数如下：“reg:linear” —— 线性回归,“reg:logistic”—— 逻辑回归。
        #'multi:softmax'
        'gamma': 0.03,                      # 最小损失减少，模型在默认情况下，对于一个节点的划分只有在其loss function 得到结果大于0的情况下才进行，而gamma 给定了所需的最低loss function的值
        'max_depth': 8,                     # 数的最大深度。缺省值为6，树的深度越大，则对数据的拟合程度越高（过拟合程度也越高）。即该参数也是控制过拟合
        'lambda': 4,                        # L2 正则的惩罚系数
        'subsample': 0.8,                   # 用于训练模型的子样本占整个样本集合的比例。如果设置为0.5则意味着XGBoost将随机的从整个样本集合中随机的抽取出50%的子样本建立树模型，这能够防止过拟合。
        'colsample_bytree': 0.8,            # 在建立树时对特征采样的比例。缺省值为1。
#         'eval_metric':'mape_ln',
        'min_child_weight': 8.5,            # 孩子节点中最小的样本权重和，在现行回归模型中，这个参数是指建立每个模型所需要的最小样本数。该成熟越大算法越conservative。即调大这个参数能够控制过拟合。
        'silent': 1,                        # 取0时表示打印出运行时信息，取1时表示以缄默方式运行，不打印运行时信息。缺省值为0。
        'eta': 0.03,                        # 为了防止过拟合，更新过程中用到的收缩步长。通常最后设置eta为0.01~0.2
        'seed': 1000}                       # 随机数的种子。缺省值为0。
    num_boost_round = 2000
    plst  = params_xgb.items()
    model_xgb = xgb.train(plst, xgb_train, num_boost_round,evals=[(xgb_train,'train'),(xgb_test,'test')],early_stopping_rounds=20,verbose_eval=True)
    return model_xgb